# Poeem Tutorial

In this notebook, we will demonstrate how to use *poeem* to jointly learn embedding index together with retrieval model. In addition to learning how to use *poeem*, you will also learn

- Write a simple embedding retrieval model
- Nearest neighbor search with brute force
- Approximate nearest neighbor (ANN) search with Faiss - a Facebook open source library for ANN search with separately already learned embeddings
- Approximate nearest neighbor (ANN) search with *Poeem* 

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import tensorflow as tf
import numpy as np
import poeem


So far, *poeem* only supports Tensorflow 1.15, other tensorflow versions have not been tested. Users may need to make minor changes accordingly to let it run on other versions of Tensorflow.

In [4]:
assert tf.__version__[:4] == '1.15'

release gpu source

In [26]:
pid = os.getpid()
!kill -9 $pid

: 

: 

## Toy data

To demonstrate how *poeem* works, here we synthesizes a toy data for a quick tutorial. More real-world and larger data set tutorial is given [here]() 

In this toy data, the query and item are both represented as numerical ID numbers, which is simply also the row indices to their embedding matrices. Specifically, 

- a query is an integer number ranging from 0 to *vocab_size* (10,000 in this tutorial)
- a item is an integer number ranging from 0 to *vocab_size* (10,000 in this tutorial)
- a query ending with last two digits as *xy*, will retrieve items ending with last 4 digits as *abcd* where any two of them equal to x and y, e.g., a=x, b=y, or c=x, b=y and so on.

In [3]:
N = 1000000  # number of training examples
VOCAB_SIZE = 10000 

In [5]:
# simulate the data
query = np.random.randint(0, high=VOCAB_SIZE, size=N)
item = np.random.randint(0, high=VOCAB_SIZE, size=N)

d, c, b, a = item % 10, (item // 10) % 10, (item // 100) % 10, (item // 1000) % 10

def get_xy(a, axis):
    idx = np.random.rand(*a.shape).argsort(axis=axis)
    shuffled = np.take_along_axis(a,idx,axis=axis)
    return shuffled[:, 0], shuffled[:, 1]

x, y = get_xy(np.stack([a, b, c, d], axis=1), 1)
query = (query // 100) * 100 + x * 10 + y

Let's take a look at the synthetic data to make sure the pattern is correct.

In [6]:
[query[:10], item[:10]]

[array([9991, 2451, 6431, 7001, 6125, 5145, 2212, 4948,  135, 4029]),
 array([1392, 9195, 2391, 3100, 2535, 1945, 2122, 5842, 6435, 2219])]

In [29]:
os.makedirs('./data', exist_ok=True)
np.save('./data/query.npy', query)
np.save('./data/item.npy', item)

## Training

In this training section, we will write a very simple embedding retrieval model for demonstration. Please be advised that this embedding model is solely for tutorial but not immediately applicable to real-world industrial systems yet where more practical techniques are necessary.

First let's define some hyperparameters

In [4]:
BATCH_SIZE = 128
LEARNING_RATE = 0.1
EPOCH = 3
EMB_DIM = 64

Second, let's leverage the Tensorflow Estimator API with custom model_fn where we can define the model by ourselves and reuse the other convenient utilities to train the model.

The queries and items are represented as separate embeddings.

In [8]:
def model_fn(features, labels, mode):
    query_column = tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            key='query',
            vocabulary_list=range(VOCAB_SIZE),
            dtype=tf.int32),
        dimension=EMB_DIM)
    item_column = tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            key='item',
            vocabulary_list=range(VOCAB_SIZE),
            dtype=tf.int32),
        dimension=EMB_DIM)
    
    query_emb = tf.feature_column.input_layer(features, [query_column])
    item_emb = tf.feature_column.input_layer(features, [item_column])

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode, predictions={'query': query_emb, 'item': item_emb})

    def cosine(a, b):
        a = tf.nn.l2_normalize(a, axis=1)
        b = tf.nn.l2_normalize(b, axis=1)
        return tf.matmul(a, b, transpose_b=True)

    scores = cosine(query_emb, item_emb)

    batch_size = tf.shape(query_emb)[0]
    loss = tf.reduce_sum(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=tf.eye(batch_size),
            logits=scores * 30))  # 1/30 is softmax temperature. Not carefully tune.

    optimizer = tf.train.AdagradOptimizer(LEARNING_RATE)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    return tf.estimator.EstimatorSpec(
        mode, loss=loss, train_op=train_op, predictions={'query': query_emb, 'item': item_emb})


Third, let's define an input function that feeds data into the *model_fn* defined above.

In [9]:
def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices({'query': query.astype(np.int32), 'item': item.astype(np.int32)})
    dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).repeat(EPOCH)
    return dataset

Finally, we are ready to train the model with the above defined *model_fn* and *input_fn* by simply two lines.

In [10]:
retrieval_model = tf.estimator.Estimator(model_fn=model_fn, model_dir='./retrieve_model')
retrieval_model.train(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './retrieve_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2ef54b4350>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Vari

2023-02-09 14:58:16.917406: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-02-09 14:58:16.950066: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2399965000 Hz
2023-02-09 14:58:16.952973: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x416f030 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-02-09 14:58:16.953030: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-02-09 14:58:16.958511: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1


INFO:tensorflow:Restoring parameters from ./retrieve_model/model.ckpt-46878
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2023-02-09 14:58:21.263156: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2e214e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-09 14:58:21.263204: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2023-02-09 14:58:21.265666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:08:00.0
2023-02-09 14:58:21.265949: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.2/lib64:/usr/lib/x86_64-linux-gnu
2023-02-09 14:58:21.266063: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; d

INFO:tensorflow:Saving checkpoints for 46878 into ./retrieve_model/model.ckpt.
INFO:tensorflow:loss = 389.53833, step = 46879
INFO:tensorflow:global_step/sec: 212.134
INFO:tensorflow:loss = 378.653, step = 46979 (0.470 sec)
INFO:tensorflow:global_step/sec: 273.598
INFO:tensorflow:loss = 365.52658, step = 47079 (0.366 sec)
INFO:tensorflow:global_step/sec: 318.078
INFO:tensorflow:loss = 367.5211, step = 47179 (0.314 sec)
INFO:tensorflow:global_step/sec: 286.807
INFO:tensorflow:loss = 392.0479, step = 47279 (0.350 sec)
INFO:tensorflow:global_step/sec: 297.639
INFO:tensorflow:loss = 383.17627, step = 47379 (0.334 sec)
INFO:tensorflow:global_step/sec: 248.569
INFO:tensorflow:loss = 379.45837, step = 47479 (0.402 sec)
INFO:tensorflow:global_step/sec: 287.358
INFO:tensorflow:loss = 389.31494, step = 47579 (0.348 sec)
INFO:tensorflow:global_step/sec: 262.147
INFO:tensorflow:loss = 358.67267, step = 47679 (0.381 sec)
INFO:tensorflow:global_step/sec: 313.637
INFO:tensorflow:loss = 375.98932, ste

## Retrieval

### Export embeddings

After the retrieval model is trained, we need to export all the query and item embeddings with the trained parameters. With Tensorflow Estimator framework, this can be done easily by constructing another *input_fn* to feed all the data to the model and grab the predictions once. 

In [7]:
def predict_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices({'query': np.arange(VOCAB_SIZE), 'item': np.arange(VOCAB_SIZE)})
    dataset = dataset.batch(VOCAB_SIZE)
    return dataset

results = list(retrieval_model.predict(input_fn=predict_input_fn))
query_emb = np.stack([r['query'] for r in results], axis=0)
item_emb = np.stack([r['item'] for r in results], axis=0)

NameError: name 'retrieval_model' is not defined

In [13]:
np.save('./data/query_emb.npy', query_emb)
np.save('./data/item_emb.npy', item_emb)

In [2]:
import torch
import numpy as np

/home/yuanrz/miniconda3/envs/torch1.9/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
query_emb = np.load('./data/query_emb.npy')
item_emb = np.load('./data/item_emb.npy')

In [4]:
query_norm = torch.nn.functional.normalize(torch.tensor(query_emb), p=2)

In [5]:
query_norm[0]

tensor([ 0.1024, -0.0735, -0.0899,  0.0190,  0.0833, -0.0045, -0.1742,  0.1705,
         0.1849,  0.1854, -0.2887,  0.0201, -0.1863,  0.0487,  0.0322, -0.1010,
        -0.0838,  0.0083,  0.2286,  0.1472,  0.0049,  0.1628, -0.0871,  0.0269,
         0.0526,  0.0511, -0.2437,  0.1787, -0.0423,  0.0207,  0.2389, -0.0704,
        -0.0608,  0.1463, -0.2012, -0.1345, -0.0100,  0.0452,  0.0235, -0.0113,
         0.0050, -0.1005, -0.0125, -0.1183, -0.1659,  0.1518,  0.0426,  0.1222,
        -0.0601,  0.1255,  0.1356, -0.0433,  0.1330,  0.0824, -0.2870,  0.0395,
        -0.0584, -0.1328,  0.2278, -0.0669, -0.0675,  0.1167, -0.0666,  0.0755])

In [8]:
query_norm2 = query_emb / np.linalg.norm(query_emb, ord=2, axis=1, keepdims=True)

In [9]:
query_norm2[0]

array([ 0.10241943, -0.07352483, -0.08994059,  0.01903564,  0.08332656,
       -0.00453188, -0.17419763,  0.17049815,  0.18494225,  0.1854223 ,
       -0.2886851 ,  0.0200959 , -0.18626961,  0.04869973,  0.03217772,
       -0.10100898, -0.08377016,  0.00828866,  0.22862867,  0.14716628,
        0.00486718,  0.162839  , -0.08708248,  0.02692058,  0.05261027,
        0.05111469, -0.24366027,  0.1786856 , -0.0422966 ,  0.02070607,
        0.23886909, -0.07043963, -0.06076782,  0.14626878, -0.20122978,
       -0.13448322, -0.0100024 ,  0.04517439,  0.02345273, -0.01131734,
        0.00497423, -0.10047087, -0.01251143, -0.11829784, -0.16593523,
        0.15178622,  0.04259186,  0.12220024, -0.06010552,  0.12549165,
        0.13559249, -0.04334374,  0.13296713,  0.08239124, -0.28702247,
        0.03951252, -0.0583694 , -0.13283312,  0.22781931, -0.06694355,
       -0.06747849,  0.11667447, -0.06663574,  0.07553485], dtype=float32)

### Brute force search

Before we try approximate nearest neighbor (ANN) search algorithms, let's first do a brute force search to get the upper bound of the retrieval accuracy. Theoretically, any ANN search algorithms should be much faster than Brute Force method but somewhat worse in retrieval accuracy.

In [31]:
def brute_force_search(query_id, items, k=10):
    query = query_emb[query_id:(query_id+1), :]
    query_norm = np.linalg.norm(query, axis=1, keepdims=True)
    item_norm = np.linalg.norm(items, axis=1, keepdims=True)
    # query_norm = query / np.linalg.norm(query, axis=1, keepdims=True)
    # item_norm = items / np.linalg.norm(items, axis=1, keepdims=True)
    # print(item_norm.shape)
    cos = np.matmul(query, np.transpose(items)) / query_norm / np.transpose(item_norm)
    cos = cos.flatten()
    sorted_item_id = np.argsort(-cos)
    return sorted_item_id[:k]

Let's have quick look at the retrieval results. Note that the retrieved item IDs all have the two digits, 8 and 9.

In [32]:
brute_force_search(98, item_emb)

array([6898, 8989, 8589, 3889, 9884, 9869, 9488, 9849, 2988, 9485])

Now we can compute a comprehensive retrieval accuracy called precision@k, where we use k=100. This metric measures that for the top 100 retrieved items, how much percentage of them are correct.

In [21]:
def precision_at_k(search_fn, query_id, items, k=100):
    nn_items = search_fn(query_id, items, k=k)
    
    d, c, b, a = nn_items % 10, (nn_items // 10) % 10, (nn_items // 100) % 10, (nn_items // 1000) % 10
    abcd = np.stack([a, b, c, d], axis=1)
    y, x = query_id % 10, (query_id // 10) % 10
    # check if x and y can be drawn from abcd without replacement
    match = np.sum(np.logical_or(x == abcd, y == abcd), axis=1) >= 2 
    precision = np.sum(match) / k
    return precision

In [13]:
precision_at_100 = [precision_at_k(brute_force_search, i, item_emb, k=100) for i in range(VOCAB_SIZE)]
print("overall precision@100 = ", np.mean(precision_at_100))

overall precision@100 =  0.9986279999999998


### HNSW

In [33]:
import hnswlib

In [34]:
query_norm = query_emb / np.linalg.norm(query_emb, ord=2, axis=1, keepdims=True)
item_norm = item_emb / np.linalg.norm(item_emb, ord=2, axis=1, keepdims=True)

In [35]:
# p = hnswlib.Index(space = 'cosine', dim=EMB_DIM)
p = hnswlib.Index(space = 'l2', dim=EMB_DIM)
p.init_index(max_elements = N, ef_construction=100, M=16)
p.set_ef(10)
p.set_num_threads(4)
p.add_items(item_emb)

In [36]:
def hnsw_search(query_id, items, k=10):
    query = query_norm[query_id:(query_id+1), :]
    item_ids, _ = p.knn_query(query, k=k)
    return item_ids[0]

In [38]:
precision_at_100 = [precision_at_k(hnsw_search, i, None, k=100) for i in range(VOCAB_SIZE)]
print("overall precision@100 = ", np.mean(precision_at_100))

overall precision@100 =  0.9999


## Faiss search

Now let's try Faiss, which is a widely used ANN library developed by Facebook. It is based on Product Quantization techniques. Here we set our index type to be 'IVF8,PQ8', which means coarse quantization into *8* clusters and then product quantization into *8* segments, each of which is represented by *one* byte, or *2^8 = 256* subvectors.

First, we need to build an embedding index with all the item embeddings. Note that we need to first normalize all item embeddings before building the index with *inner product* as distance metric, or more precisely, inverse distance metric.

In [20]:
import faiss

index = faiss.index_factory(EMB_DIM, 'IVF8,PQ8', faiss.METRIC_INNER_PRODUCT)
item_emb = item_emb / np.linalg.norm(item_emb, axis=1, keepdims=True)
index.train(item_emb)
index.add(item_emb)

In [22]:
def faiss_search(query_id, items, k=10):
    query = query_emb[query_id:(query_id+1), :]
    D, I = index.search(query, k)
    D, I = D.flatten(), I.flatten()
    return I[:k]

In [23]:
precision_at_100 = [precision_at_k(faiss_search, i, item_emb, k=100) for i in range(VOCAB_SIZE)]
print("overall precision@100 = ", np.mean(precision_at_100))   

overall precision@100 =  0.961116


## Poeem search

Apart from above separately built embedding index, *poeem* learns the embedding model jointly with embedding index. Thus, there is no extra index building step. But we need to make some simple changes into the above *model_fn* function to adopt *poeem* indexing layer.

Note that we only need to make changes at three places, marked by ### [poeem code]

In [13]:
def poeem_model_fn(features, labels, mode, params):
    query_column = tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            key='query',
            vocabulary_list=range(VOCAB_SIZE),
            dtype=tf.int32),
        dimension=EMB_DIM)
    query_emb = tf.feature_column.input_layer(features, [query_column])
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        ### [poeem code] directly do ANN search in the model as a TensorFlow op.
        if params.get('item_search', False):
            index = poeem.search.index_from_file(params['index_file'])
            neighbors, scores = index.search(
                tf.expand_dims(query_emb, 1),
                params['topk'],
                params['nprobe'],
                params['metric_type'],
                verbose=False)        
            return tf.estimator.EstimatorSpec(
                mode, predictions={'neighbors': neighbors, 'scores': scores})
        ### end [poeem code]
        
    item_column = tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            key='item',
            vocabulary_list=range(VOCAB_SIZE),
            dtype=tf.int32),
        dimension=EMB_DIM)
    item_emb = tf.feature_column.input_layer(features, [item_column])
    item_emb = tf.nn.l2_normalize(item_emb, axis=1)
    

    ### [poeem code] item indexing layer as the last layer in item tower
    hparams = poeem.embedding.PoeemHparam(coarse_K=8,
                                          K=256,
                                          D=8,
                                          rotate=0) # exactly the same parameters as Faiss, specified above.
    item_batch_quantized = poeem.embedding.PoeemEmbed(
        EMB_DIM,
        warmup_steps=16384,
        buffer_size=8192,
        hparams=hparams,
        mode=mode)
    
    # gradient straight-through estimator. For details, check out our paper.
    item_emb_tau, coarse_code, code, regularizer = item_batch_quantized.forward(item_emb)
    item_emb = item_emb - tf.stop_gradient(item_emb - item_emb_tau)
    ### end [poeem code] 

    if mode == tf.estimator.ModeKeys.PREDICT:
        ### [poeem code] exprt item embeddings/PQ code for disk persistency.
        if params.get('item_predict', False):
            return tf.estimator.EstimatorSpec(
                mode, predictions={
                    'item_coarse_code': coarse_code,
                    'item_code': code,
                    'item_norm': tf.norm(item_emb, axis=1)
                })
        ### end [poeem code] 

    def cosine(a, b):
        a = tf.nn.l2_normalize(a, axis=1)
        b = tf.nn.l2_normalize(b, axis=1)
        return tf.matmul(a, b, transpose_b=True)

    scores = cosine(query_emb, item_emb)

    batch_size = tf.shape(query_emb)[0]
    loss = tf.reduce_sum(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=tf.eye(batch_size),
            logits=scores * 30))
    
    loss = loss + regularizer

    optimizer = tf.train.AdagradOptimizer(LEARNING_RATE)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    return tf.estimator.EstimatorSpec(
        mode, loss=loss, train_op=train_op, predictions={'query': query_emb, 'item': item_emb})


In [14]:
MODEL_DIR = './poeem_model'
poeem_model = tf.estimator.Estimator(model_fn=poeem_model_fn, model_dir=MODEL_DIR, params={})
poeem_model.train(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './poeem_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f901f8c3cd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

Instructions for updatin

2023-02-08 20:20:17.456944: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-02-08 20:20:17.486003: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2399965000 Hz
2023-02-08 20:20:17.489437: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1242c70 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-02-08 20:20:17.489509: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-02-08 20:20:17.494234: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1


INFO:tensorflow:Running local_init_op.


2023-02-08 20:20:19.493871: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x43f8f30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-08 20:20:19.493948: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2023-02-08 20:20:19.493962: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla V100-PCIE-32GB, Compute Capability 7.0
2023-02-08 20:20:19.493973: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): Tesla V100-PCIE-32GB, Compute Capability 7.0
2023-02-08 20:20:19.493984: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (3): Tesla V100-PCIE-32GB, Compute Capability 7.0
2023-02-08 20:20:19.513571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./poeem_model/model.ckpt.
INFO:tensorflow:loss = 1334.2151, step = 1
INFO:tensorflow:global_step/sec: 147.884
INFO:tensorflow:loss = 1322.4939, step = 101 (0.678 sec)
INFO:tensorflow:global_step/sec: 255.175
INFO:tensorflow:loss = 1300.1631, step = 201 (0.392 sec)
INFO:tensorflow:global_step/sec: 256.458
INFO:tensorflow:loss = 1375.9381, step = 301 (0.390 sec)
INFO:tensorflow:global_step/sec: 253.084
INFO:tensorflow:loss = 1374.4829, step = 401 (0.396 sec)
INFO:tensorflow:global_step/sec: 260.627
INFO:tensorflow:loss = 1325.1431, step = 501 (0.382 sec)
INFO:tensorflow:global_step/sec: 252.307
INFO:tensorflow:loss = 1358.9294, step = 601 (0.397 sec)
INFO:tensorflow:global_step/sec: 259.543
INFO:tensorflow:loss = 1344.8783, step = 701 (0.386 sec)
INFO:tensorflow:global_step/sec: 248.012
INFO:tensorflow:loss = 1297.4072, step = 801 (0.404 sec)
INFO:tensorflow:global_step/sec: 252.544
INFO:tensorflow:

2023-02-08 20:21:28.948156: I cpp/clustering_raw_op.cc:128] Converged at iter 55 : average distance = 0.925711, assignment changed = 79 (0.00964355), assignment frequency histogram = (873, 1) (907, 1) (940, 1) (957, 1) (997, 1) (1029, 1) (1034, 1) (1455, 1) 
2023-02-08 20:21:28.990714: I cpp/clustering_raw_op.cc:128] Converged at iter 21 : average distance = 0.177143, assignment changed = 71 (0.00866699), assignment frequency histogram = (4, 1) (14, 1) (16, 1) (18, 1) (19, 1) (20, 2) (22, 1) (23, 1) (24, 3) (25, 1) (26, 3) (27, 2) (28, 3) (29, 3) (30, 4) (31, 3) (32, 8) (33, 8) (34, 6) (35, 7) (36, 9) (37, 11) (38, 18) (39, 8) (40, 6) (41, 12) (42, 10) (43, 8) (44, 7) (45, 3) (46, 11) (47, 7) (48, 6) (49, 4) (50, 4) (51, 4) (52, 3) (53, 4) (54, 2) (55, 3) (56, 1) (57, 1) (59, 3) (64, 1) (70, 1) 
2023-02-08 20:21:29.006979: I cpp/clustering_raw_op.cc:128] Converged at iter 28 : average distance = 0.174805, assignment changed = 81 (0.0098877), assignment frequency histogram = (2, 1) (10,

INFO:tensorflow:global_step/sec: 101.166
INFO:tensorflow:loss = 459.334, step = 16501 (0.989 sec)
INFO:tensorflow:global_step/sec: 199.458
INFO:tensorflow:loss = 473.97308, step = 16601 (0.501 sec)
INFO:tensorflow:global_step/sec: 201.066
INFO:tensorflow:loss = 487.33423, step = 16701 (0.498 sec)
INFO:tensorflow:global_step/sec: 201.851
INFO:tensorflow:loss = 452.42444, step = 16801 (0.496 sec)
INFO:tensorflow:global_step/sec: 196.036
INFO:tensorflow:loss = 451.08435, step = 16901 (0.510 sec)
INFO:tensorflow:global_step/sec: 197.364
INFO:tensorflow:loss = 464.23026, step = 17001 (0.506 sec)
INFO:tensorflow:global_step/sec: 194.349
INFO:tensorflow:loss = 451.41385, step = 17101 (0.516 sec)
INFO:tensorflow:global_step/sec: 193.491
INFO:tensorflow:loss = 462.97806, step = 17201 (0.516 sec)
INFO:tensorflow:global_step/sec: 194.576
INFO:tensorflow:loss = 442.48016, step = 17301 (0.514 sec)
INFO:tensorflow:global_step/sec: 195.844
INFO:tensorflow:loss = 452.3904, step = 17401 (0.511 sec)
INF

### Export item embedding and build index

Though theoretically *poeem* does not need to build an index, we can still optionally build one to persist the embedding index into disk. Since *poeem* indexing layer has already done coarse quantization and product quantization internally, the index building just needs to export those values into an index file as follows.

In [15]:
def predict_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices({'query': np.arange(VOCAB_SIZE), 'item': np.arange(VOCAB_SIZE)})
    dataset = dataset.batch(VOCAB_SIZE)
    return dataset

poeem_model = tf.estimator.Estimator(model_fn=poeem_model_fn, model_dir=MODEL_DIR, 
                                     params={'item_predict': True})
results = list(poeem_model.predict(input_fn=predict_input_fn))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './poeem_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8fd3aa2790>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done call

2023-02-08 20:22:09.642009: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:04:00.0
2023-02-08 20:22:09.643895: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:05:00.0
2023-02-08 20:22:09.645660: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 2 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:08:00.0
2023-02-08 20:22:09.646428: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 3 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:85:00.0
2023-02-08 20:22:09.646656: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dler

Collect all the data we need to write into an index file

In [16]:
item_coarse_code = np.array([e['item_coarse_code'] for e in results])
item_code = np.array([e['item_code'] for e in results])
item_norm = np.array([e['item_norm'] for e in results])
item_id = np.arange(VOCAB_SIZE)
coarse_codebook = tf.train.load_variable(MODEL_DIR, 'coarse_centroids')
codebook = tf.train.load_variable(MODEL_DIR, 'centroids_k')

INDEX_FILE = './poeem.idx'
poeem.indexing.write_index_file(INDEX_FILE, codebook, item_id, item_norm, item_code, 
                                coarse_codebook, item_coarse_code, use_residual=True)

### Poeem nearest neighbor search

*poeem* ANN search would be in an end-to-end fashion, i.e., input a query and output its nearest neighbor items directly. The most simplest setup would be as follows 

In [17]:
poeem_model = tf.estimator.Estimator(model_fn=poeem_model_fn, model_dir=MODEL_DIR, 
                                     params={'item_search': True, 'index_file': INDEX_FILE, 
                                             'topk': 100, 'nprobe': 1, 'metric_type': 0})

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './poeem_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8fd2235fd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


To measure *poeem* rerieval accuracy, we first compute retrieval results for all queries.

In [18]:
def search_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices({'query': np.arange(VOCAB_SIZE)})
    dataset = dataset.batch(VOCAB_SIZE)
    return dataset

results = list(poeem_model.predict(input_fn=search_input_fn))
neighbors = np.array([e['neighbors'] for e in results])
scores = np.array([e['scores'] for e in results])

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./poeem_model/model.ckpt-23439
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2023-02-08 20:22:25.571497: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:04:00.0
2023-02-08 20:22:25.573328: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:05:00.0
2023-02-08 20:22:25.575088: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 2 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:08:00.0
2023-02-08 20:22:25.576005: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 3 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:85:00.0
2023-02-08 20:22:25.576286: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dler

Follows the search function interface as above Brute Force and Faiss search, so we can reuse the precision_at_k utitity function.

In [19]:
def poeem_search(query_id, items, k=100):
    return neighbors[query_id, :k]

In [22]:
precision_at_100 = [precision_at_k(poeem_search, i, None, k=100) for i in range(VOCAB_SIZE)]
print("overall precision@100 = ", np.mean(precision_at_100))   

overall precision@100 =  0.9814590000000002


**Observation**: Note that Poeem could reach higher retrieval accuracy than Faiss, by jointly learning the embedding index and retrieval model.

This is a simple example as a quick rampup for beginners. For more rigorous experimental results to draw conclusions, please checkout our SIGIR paper.